# Auteurs : BAUDET Quentin & LARMAILLARD-NOIREN Joris

Dans ce notebook, nous allons procéder à un pré-traitement approfondi des données issues du dataset de smartphones disponible sur Kaggle. L’objectif est de nettoyer, transformer et enrichir ces données afin de construire un jeu de données final, adapté à la conception d’un modèle d’intelligence artificielle dédié à la recommandation de smartphones.

Ce travail préparatoire est essentiel pour garantir la qualité des entrées du modèle, optimiser ses performances et assurer des recommandations pertinentes. Nous détaillerons chaque étape du traitement des données, depuis la gestion des valeurs manquantes jusqu’à la sélection des caractéristiques les plus informatives.

### Chargement des données

In [1]:
### Importation des modules
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math

In [2]:
### Chargement des données
data = pd.read_csv('../data/raw/Smartphones_cleaned_dataset_completed.csv', sep=';')

data.head()

,brand_name,model,price,rating,has_5g,has_nfc,has_ir_blaster,processor_brand,num_cores,processor_speed,...,refresh_rate,num_rear_cameras,num_front_cameras,os,primary_camera_rear,primary_camera_front,extended_memory_available,extended_upto,resolution_width,resolution_height
0,apple,Apple iPhone 11,38999,73.0,False,True,False,bionic,6,2.65,...,60,2,1.0,ios,12.0,12.0,0,NaN,828,1792
1,apple,Apple iPhone 11 (128GB),46999,75.0,False,True,False,bionic,6,2.65,...,60,2,1.0,ios,12.0,12.0,0,NaN,828,1792
2,apple,Apple iPhone 11 Pro Max,109900,77.0,False,True,False,bionic,6,2.65,...,60,3,1.0,ios,12.0,12.0,0,NaN,1242,2688
3,apple,Apple iPhone 12,51999,74.0,True,True,False,bionic,6,3.10,...,60,2,1.0,ios,12.0,12.0,0,NaN,1170,2532
4,apple,Apple iPhone 12 (128GB),55999,75.0,True,True,False,bionic,6,3.10,...,60,2,1.0,ios,12.0,12.0,0,NaN,1170,2532


### Traitement des données

**Changement de la devise**

Comme nous l'avons vu précédemment dans le notebook `01_exploration`, nous allons changer la devise présente dans la colonne prix, pour la convertir en euro (€), car le dataset a été construit en Inde, et il faut donc adapter la devise utilisée en Europe.

In [3]:
### Conversion des prix roupie -> euro
data['price'] = data['price'] * 0.009966

### Arrondi des prix
data['price'] = data['price'].round(2)

data = data.rename(columns={'price': 'price (€)'})

data

,brand_name,model,price (€),rating,has_5g,has_nfc,has_ir_blaster,processor_brand,num_cores,processor_speed,...,refresh_rate,num_rear_cameras,num_front_cameras,os,primary_camera_rear,primary_camera_front,extended_memory_available,extended_upto,resolution_width,resolution_height
0,apple,Apple iPhone 11,388.66,73.0,False,True,False,bionic,6,2.65,...,60,2,1.0,ios,12.0,12.0,0,NaN,828,1792
1,apple,Apple iPhone 11 (128GB),468.39,75.0,False,True,False,bionic,6,2.65,...,60,2,1.0,ios,12.0,12.0,0,NaN,828,1792
2,apple,Apple iPhone 11 Pro Max,1095.26,77.0,False,True,False,bionic,6,2.65,...,60,3,1.0,ios,12.0,12.0,0,NaN,1242,2688
3,apple,Apple iPhone 12,518.22,74.0,True,True,False,bionic,6,3.10,...,60,2,1.0,ios,12.0,12.0,0,NaN,1170,2532
4,apple,Apple iPhone 12 (128GB),558.09,75.0,True,True,False,bionic,6,3.10,...,60,2,1.0,ios,12.0,12.0,0,NaN,1170,2532
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
974,xiaomi,Xiaomi Redmi Note 9 Pro,139.51,75.0,False,False,True,snapdragon,8,2.30,...,60,4,1.0,android,48.0,16.0,1,512.0,1080,2400
975,xiaomi,Xiaomi Redmi Note 9 Pro (4GB RAM + 128GB),143.90,77.0,False,False,True,snapdragon,8,2.30,...,60,4,1.0,android,48.0,16.0,1,512.0,1080,2400
976,xiaomi,Xiaomi Redmi Note 9 Pro Max,164.34,80.0,False,False,True,snapdragon,8,2.30,...,60,4,1.0,android,64.0,32.0,1,512.0,1080,2400
977,zte,ZTE Axon 30S,199.31,82.0,True,True,False,snapdragon,8,3.20,...,120,4,1.0,android,50.0,16.0,1,NaN,1080,2460


**Suppression de colonnes**

Dans cette section, nous allons supprimer certaines colonnes qui ne nous semblent pas pertinentes, ou déterminantes pour le choix d'un téléphone, notamment des fonctionnalités qui sont gadgets comme `has_ir_blaster`. Cela permettra d'être plus orienté vers la satisfaction de la clientèle.

In [4]:
### Suppression des colonnes
data = data.drop(columns=['has_ir_blaster', 'has_nfc'])

**Critère d'évaluation de l'autonomie d'un téléphone**

Maintenant, nous allons rajouter une étiquette à chaque téléphone pour évaluer le niveau d'autonomie de chaque téléphone, dans une démarche de satisfaction client. Si un client souhaite un téléphone avec une bonne autonomie, et rien d'autre cela permettra de mieux l'orienter vers ce qu'il recherche.

In [5]:
### Fonction d'étiquetage des téléphones par rapport à leur autonomie
def evaluate_battery_capacity(capacity) -> str:
    """
    Attribue une étiquette à chaque téléphone sur le seul critère de la capacité de leur batterie.
    
    :param capacity: Capacité du téléphone
    :return: Étiquette finale
    """
    
    if capacity < 3500:
        return "faible"
    elif capacity < 4500:
        return "moyenne"
    else:
        return "haute"

In [6]:
### Application de la fonction
data['quality_battery_autonomy'] = data['battery_capacity'].apply(evaluate_battery_capacity)

**Création d'une fonction de scoring pour catégoriser les téléphones selon leur performance**

Dans la section suivante, nous allons implémenter une fonction dans laquelle chaque ligne du dataset sera analysée, et pour chaque attribut des téléphones, nous allons attribuer un score qui sera utilisé après pour calculer un score final qui déterminera la catégorie du téléphone. Cela permettra d'affiner les futures recommandations du modèle, et de fournir des résultats qui pourront répondre aux besoins du client.

In [7]:
### Importation module
from typing import List, Tuple

### Fonction qui calcule un score total de chaque téléphone
def compute_total_score(phone) -> float:
    """
    Calcule un score de performance pour un téléphone donné 
    en fonction de ses caractéristiques techniques.

    Ce score permet ensuite de classifier le téléphone 
    en haute performance, performance intermédiaire ou basique.
    :param phone: Correspond au téléphone (ligne) que l'on considère actuellement pour le calcul du score
    :return: Le score final du téléphone en fonction des différents critères d'évaluation
    """
    ### Fonction imbriquée qui généralise la logique de calcul des scores : pour les colonnes numériques
    def score_range(val, seuils: List[Tuple[float, float]]) -> float:
        for seuil, point in seuils:
            if val >= seuil:
                return point
        return 0
    
    ### Fonction imbriquée qui généralise la logique de calcul des scores : pour les colonnes booléennes
    def score_boolean(val, point_if_true=0.5) -> float:
        return point_if_true if val else 0
    
    ### Variable qui stocke le score final
    score: float = 0
    
    ### Poids de chaque caractéristique des téléphones attribués selon l'importance de la caractéristique pour le choix du téléphone
    weights: List[float] = [0.10, 0.10, 0.20, 0.10, 0.15, 0.10, 0.10, 0.10, 0.05]
    
    ### Calcul des scores : colonnes numériques
    score += score_range(phone['rating'], [(85, 1), (80, 0.5)]) * weights[0]
    score += score_range(phone['num_cores'], [(8, 1), (6, 0.5)]) * weights[1]
    score += score_range(phone['processor_speed'], [(2.8, 1), (2.4, 0.5)]) * weights[2]
    score += score_range(phone['battery_capacity'], [(4500, 1), (3500, 0.5)]) * weights[3]
    score += score_range(phone['ram_capacity'], [(12, 1), (8, 0.5)]) * weights[4]
    score += score_range(phone['internal_memory'], [(256, 1), (128, 0.5)]) * weights[5]
    score += score_range(phone['refresh_rate'], [(120, 1), (90, 0.5)]) * weights[6]
    score += score_range(phone['primary_camera_rear'], [(64, 1), (48, 0.5)]) * weights[7]
    
    ### Calcul des scores : colonnes booléennes
    score += score_boolean(phone['has_5g'], 1) * weights[-1]
    
    return score
        

Nous pouvons voir également que la colonne `rating` possède des valeurs manquantes qui sont cruciales pour le calcul du score de chaque téléphone. Pour remplacer ces valeurs, nous allons calculer les valeurs médianes des `rating` par marque de téléphones, et les attribuer aux téléphones qui n'ont pas de note répertoriée.

In [8]:
### Calcul des médianes des `rating` par marque
median_per_brand = data.groupby('brand_name')['rating'].median()

data['rating'] = data.apply(
    lambda phone: median_per_brand[phone['brand_name']] if pd.isna(phone['rating']) else phone['rating'],
    axis=1
)

In [9]:
### Fonction de catégorisation des téléphones
def determine_performance(phone) -> str:
    """
    Calcule un score de performance pour un téléphone donné 
    en fonction de ses caractéristiques techniques.

    Ce score permet ensuite de classifier le téléphone 
    en haut de gamme, milieu de gamme ou bas de gamme.
     
    :param phone: Prend en argument chaque téléphone (ligne) du dataset
    :return: Une chaîne de caractères qui indique la catégorie du téléphone
    """
    score: float = compute_total_score(phone)
    
    ### Attribution des catégories
    if score >= 0.7:
        return "haut performance"
    elif score >= 0.4:
        return "performance intermédiaire"
    else:
        return "basique"

In [10]:
### Application au dataset
data['performance category'] = data.apply(determine_performance, axis=1)

data

,brand_name,model,price (€),rating,has_5g,processor_brand,num_cores,processor_speed,battery_capacity,fast_charging_available,...,num_front_cameras,os,primary_camera_rear,primary_camera_front,extended_memory_available,extended_upto,resolution_width,resolution_height,quality_battery_autonomy,performance category
0,apple,Apple iPhone 11,388.66,73.0,False,bionic,6,2.65,3110,0,...,1.0,ios,12.0,12.0,0,NaN,828,1792,faible,basique
1,apple,Apple iPhone 11 (128GB),468.39,75.0,False,bionic,6,2.65,3110,0,...,1.0,ios,12.0,12.0,0,NaN,828,1792,faible,basique
2,apple,Apple iPhone 11 Pro Max,1095.26,77.0,False,bionic,6,2.65,3500,1,...,1.0,ios,12.0,12.0,0,NaN,1242,2688,moyenne,basique
3,apple,Apple iPhone 12,518.22,74.0,True,bionic,6,3.10,2815,0,...,1.0,ios,12.0,12.0,0,NaN,1170,2532,faible,basique
4,apple,Apple iPhone 12 (128GB),558.09,75.0,True,bionic,6,3.10,2815,0,...,1.0,ios,12.0,12.0,0,NaN,1170,2532,faible,basique
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
974,xiaomi,Xiaomi Redmi Note 9 Pro,139.51,75.0,False,snapdragon,8,2.30,5020,1,...,1.0,android,48.0,16.0,1,512.0,1080,2400,haute,basique
975,xiaomi,Xiaomi Redmi Note 9 Pro (4GB RAM + 128GB),143.90,77.0,False,snapdragon,8,2.30,5020,1,...,1.0,android,48.0,16.0,1,512.0,1080,2400,haute,basique
976,xiaomi,Xiaomi Redmi Note 9 Pro Max,164.34,80.0,False,snapdragon,8,2.30,5020,1,...,1.0,android,64.0,32.0,1,512.0,1080,2400,haute,basique
977,zte,ZTE Axon 30S,199.31,82.0,True,snapdragon,8,3.20,4200,1,...,1.0,android,50.0,16.0,1,NaN,1080,2460,moyenne,performance intermédiaire


**Estimation du rapport qualité/prix**

Pour prendre en compte l'avis des utilisateurs sur les téléphones par rapport au prix de vente qu'ils ont, nous allons créer une nouvelle métrique `quality-price ratio`. Nous avons vu que pour une même note, par exemple 85, les prix peuvent varier de moins de `100 €` à plus de `1000 €`, ce qui montre que la note ne reflète pas uniquement le prix, mais elle peut aussi intégrer la qualité perçue, la marque, la fiabilité, ou encore l’expérience utilisateur.

Cependant, pour définir cette métrique, nous allons l'appliquer par segment de gamme. C'est-à-dire qu'un téléphone de haute gamme peut avoir, avec cette nouvelle métrique, un ratio qui peut être bas. Et, nous devons noter que, les téléphones haut de gamme ne sont pas destinés à être économique, il n'est pas pertinent d'estimer ce rapport pour cette gamme de téléphone. Donc, pour ce faire, nous allons créer des ratios par gamme. 

Alors, nous allons utiliser la proposition suivante de segmentation des prix :
- Haut de gamme : > 700 €
- Milieu de gamme : 300 - 700 €
- Entrée de gamme : < 300 €

Nous allons créer une nouvelle colonne qui recense le segment du prix auquel appartient chaque téléphone.

In [11]:
### Fonction de segmentation des prix
def segmenting_prices(price : float) -> str:
    """
    Détermine le segment dans lequel un téléphone se trouve à partir de son prix.
    
    :param price: Prix du téléphone
    :return: Segment auquel il appartient
    """
    if price < 300:
        return "bas"
    elif price <= 700:
        return "milieu"
    else:
        return "haut"

In [12]:
### Application de la fonction de segmentation
data['price_segment'] = data['price (€)'].apply(segmenting_prices)

Nous allons maintenant calculer le ratio `rating / price`, mais étant donné qu'il n'est pas comparable entre les différents segments, nous allons ensuite normaliser le ratio dans chaque segment. Cela donne un score de 0 à 1, relatif à la performance du téléphone dans sa gamme de prix. Cela nous permettra d'avoir des choses comme, un téléphone à 200 € avec 80 de rating et un à 1000 € avec 90 de rating ayant des scores similaires dans leur segment respectif, car chacun est bon dans sa catégorie.

In [13]:
### Calcul du ratio qualité / prix
data['ratio_rating_price'] = data['rating'] / data['price (€)']

In [14]:
### Normalisation des ratios dans chaque segment de prix
data['ratio_segment'] = data.groupby('price_segment')['ratio_rating_price'].transform(
    lambda x: (x - x.min()) / (x.max() - x.min())
)

data

,brand_name,model,price (€),rating,has_5g,processor_brand,num_cores,processor_speed,battery_capacity,fast_charging_available,...,primary_camera_front,extended_memory_available,extended_upto,resolution_width,resolution_height,quality_battery_autonomy,performance category,price_segment,ratio_rating_price,ratio_segment
0,apple,Apple iPhone 11,388.66,73.0,False,bionic,6,2.65,3110,0,...,12.0,0,NaN,828,1792,faible,basique,milieu,0.187825,0.479802
1,apple,Apple iPhone 11 (128GB),468.39,75.0,False,bionic,6,2.65,3110,0,...,12.0,0,NaN,828,1792,faible,basique,milieu,0.160123,0.327847
2,apple,Apple iPhone 11 Pro Max,1095.26,77.0,False,bionic,6,2.65,3500,1,...,12.0,0,NaN,1242,2688,moyenne,basique,haut,0.070303,0.528967
3,apple,Apple iPhone 12,518.22,74.0,True,bionic,6,3.10,2815,0,...,12.0,0,NaN,1170,2532,faible,basique,milieu,0.142796,0.232805
4,apple,Apple iPhone 12 (128GB),558.09,75.0,True,bionic,6,3.10,2815,0,...,12.0,0,NaN,1170,2532,faible,basique,milieu,0.134387,0.186675
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
974,xiaomi,Xiaomi Redmi Note 9 Pro,139.51,75.0,False,snapdragon,8,2.30,5020,1,...,16.0,1,512.0,1080,2400,haute,basique,bas,0.537596,0.208098
975,xiaomi,Xiaomi Redmi Note 9 Pro (4GB RAM + 128GB),143.90,77.0,False,snapdragon,8,2.30,5020,1,...,16.0,1,512.0,1080,2400,haute,basique,bas,0.535094,0.206536
976,xiaomi,Xiaomi Redmi Note 9 Pro Max,164.34,80.0,False,snapdragon,8,2.30,5020,1,...,32.0,1,512.0,1080,2400,haute,basique,bas,0.486796,0.176399
977,zte,ZTE Axon 30S,199.31,82.0,True,snapdragon,8,3.20,4200,1,...,16.0,1,NaN,1080,2460,moyenne,performance intermédiaire,bas,0.411419,0.129366


Voici une petite analyse de la distribution du ratio segmenté :

* Segment bas (< 300 €) :
    * `Moyenne : 0.21`
    * `25% des modèles ont un score < 0.11`
    * `Score élevé (75e percentile) : > 0.27`
    * Les bons scores sont rares dans ce segment, les valeurs sont globalement faibles.

* Segment milieu (300–700 €) :
    * `Moyenne : 0.51`
    * `25% des modèles : < 0.28`
    * `75% des modèles : < 0.75`
    * Cette distribution est plus équilibrée, nous avons de bons segments pour identifier des rapports qualités-prix corrects.

* Segment haut (> 700 €) :
    * `Moyenne : 0.61`
    * `25% des modèles : < 0.46`
    * `75% des modèles : > 0.77`
    * Ici, nous avons plus de variance, et certains modèles haut de gamme sont bien notés pour leur prix.

Nous allons donc écrire une fonction nous permettant d'attribuer nos étiquettes `qualité/prix` selon des seuils basés sur l'analyse des distributions des différents segments.

In [15]:
### Fonction d'étiquetage
def label_ratio(phone) -> str:
    """
    Fonction d'attribution de label sur le rapport qualité/prix en fonction du segment dans lequel se trouve le téléphone considéré.
    
    :param phone: Téléphone à étiqueter sur son rapport qualité/prix
    :return: Étiquette finale
    """
    
    ### Récupération des ratios
    val = phone['ratio_segment']
    segment = phone['price_segment']
    
    ### Disjonction de cas et attribution des labels
    ### Segment bas
    if segment == 'bas':
        if val >= 0.30:
            return "excellent"
        elif val >= 0.20:
            return "bon"
        elif val >= 0.10:
            return "moyen"
        else:
            return "faible"
        
    ### Segment milieu
    elif segment == 'milieu':
        if val >= 0.70:
            return "excellent"
        elif val >= 0.50:
            return "bon"
        elif val >= 0.30:
            return "moyen"
        else:
            return "faible"
    
    ### Segment haut
    elif segment == 'haut':
        if val >= 0.80:
            return "excellent"
        elif val >= 0.60:
            return "bon"
        elif val >= 0.40:
            return "moyen"
        else:
            return "faible"
    
    ### Renvoyer inconnu par mesure de sécurité (Possibles valeurs manquantes qui ne peuvent pas être remplacées)
    return "inconnu"

In [16]:
### Application de la fonction d'étiquetage
data['quality-price_ratio'] = data.apply(label_ratio, axis=1)

data

,brand_name,model,price (€),rating,has_5g,processor_brand,num_cores,processor_speed,battery_capacity,fast_charging_available,...,extended_memory_available,extended_upto,resolution_width,resolution_height,quality_battery_autonomy,performance category,price_segment,ratio_rating_price,ratio_segment,quality-price_ratio
0,apple,Apple iPhone 11,388.66,73.0,False,bionic,6,2.65,3110,0,...,0,NaN,828,1792,faible,basique,milieu,0.187825,0.479802,moyen
1,apple,Apple iPhone 11 (128GB),468.39,75.0,False,bionic,6,2.65,3110,0,...,0,NaN,828,1792,faible,basique,milieu,0.160123,0.327847,moyen
2,apple,Apple iPhone 11 Pro Max,1095.26,77.0,False,bionic,6,2.65,3500,1,...,0,NaN,1242,2688,moyenne,basique,haut,0.070303,0.528967,moyen
3,apple,Apple iPhone 12,518.22,74.0,True,bionic,6,3.10,2815,0,...,0,NaN,1170,2532,faible,basique,milieu,0.142796,0.232805,faible
4,apple,Apple iPhone 12 (128GB),558.09,75.0,True,bionic,6,3.10,2815,0,...,0,NaN,1170,2532,faible,basique,milieu,0.134387,0.186675,faible
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
974,xiaomi,Xiaomi Redmi Note 9 Pro,139.51,75.0,False,snapdragon,8,2.30,5020,1,...,1,512.0,1080,2400,haute,basique,bas,0.537596,0.208098,bon
975,xiaomi,Xiaomi Redmi Note 9 Pro (4GB RAM + 128GB),143.90,77.0,False,snapdragon,8,2.30,5020,1,...,1,512.0,1080,2400,haute,basique,bas,0.535094,0.206536,bon
976,xiaomi,Xiaomi Redmi Note 9 Pro Max,164.34,80.0,False,snapdragon,8,2.30,5020,1,...,1,512.0,1080,2400,haute,basique,bas,0.486796,0.176399,moyen
977,zte,ZTE Axon 30S,199.31,82.0,True,snapdragon,8,3.20,4200,1,...,1,NaN,1080,2460,moyenne,performance intermédiaire,bas,0.411419,0.129366,moyen


**Enregistrement du dataframe**

Dans un premier temps, nous allons réorganiser les colonnes, et supprimer certaines dont nous n'aurons pas besoin pour la mise en place de notre modèle.

In [17]:
### Nouvel ordre des colonnes
new_order = ['brand_name', 'model', 'price (€)', 'price_segment','rating', 'ratio_rating_price', 'ratio_segment', 'quality-price_ratio', 'has_5g', 'processor_brand', 'num_cores', 'processor_speed', 'battery_capacity', 'quality_battery_autonomy', 'fast_charging_available', 'fast_charging', 'ram_capacity', 'internal_memory', 'screen_size', 'refresh_rate', 'num_rear_cameras', 'num_front_cameras', 'os', 'primary_camera_rear', 'primary_camera_front', 'extended_memory_available', 'extended_upto', 'resolution_width', 'resolution_height', 'performance category']

data = data[new_order]

In [18]:
data

,brand_name,model,price (€),price_segment,rating,ratio_rating_price,ratio_segment,quality-price_ratio,has_5g,processor_brand,...,num_rear_cameras,num_front_cameras,os,primary_camera_rear,primary_camera_front,extended_memory_available,extended_upto,resolution_width,resolution_height,performance category
0,apple,Apple iPhone 11,388.66,milieu,73.0,0.187825,0.479802,moyen,False,bionic,...,2,1.0,ios,12.0,12.0,0,NaN,828,1792,basique
1,apple,Apple iPhone 11 (128GB),468.39,milieu,75.0,0.160123,0.327847,moyen,False,bionic,...,2,1.0,ios,12.0,12.0,0,NaN,828,1792,basique
2,apple,Apple iPhone 11 Pro Max,1095.26,haut,77.0,0.070303,0.528967,moyen,False,bionic,...,3,1.0,ios,12.0,12.0,0,NaN,1242,2688,basique
3,apple,Apple iPhone 12,518.22,milieu,74.0,0.142796,0.232805,faible,True,bionic,...,2,1.0,ios,12.0,12.0,0,NaN,1170,2532,basique
4,apple,Apple iPhone 12 (128GB),558.09,milieu,75.0,0.134387,0.186675,faible,True,bionic,...,2,1.0,ios,12.0,12.0,0,NaN,1170,2532,basique
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
974,xiaomi,Xiaomi Redmi Note 9 Pro,139.51,bas,75.0,0.537596,0.208098,bon,False,snapdragon,...,4,1.0,android,48.0,16.0,1,512.0,1080,2400,basique
975,xiaomi,Xiaomi Redmi Note 9 Pro (4GB RAM + 128GB),143.90,bas,77.0,0.535094,0.206536,bon,False,snapdragon,...,4,1.0,android,48.0,16.0,1,512.0,1080,2400,basique
976,xiaomi,Xiaomi Redmi Note 9 Pro Max,164.34,bas,80.0,0.486796,0.176399,moyen,False,snapdragon,...,4,1.0,android,64.0,32.0,1,512.0,1080,2400,basique
977,zte,ZTE Axon 30S,199.31,bas,82.0,0.411419,0.129366,moyen,True,snapdragon,...,4,1.0,android,50.0,16.0,1,NaN,1080,2460,performance intermédiaire


**Extraction du dataset traîté**

In [19]:
data.to_csv('../data/processed/Smartphones_cleaned_dataset_processed.csv', index=False)